In [25]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

# 使用给定的原图和渲染图路径
original_image_paths = [
    r"C:\Users\27118\Desktop\pics2\chess_orig.png", 
    r"C:\Users\27118\Desktop\pics2\office_orig.png", 
    r"C:\Users\27118\Desktop\pics2\redkitchen_orig.png"
]

rendered_image_paths = [
    r"C:\Users\27118\Desktop\pics2\chess_loc.png", 
    r"C:\Users\27118\Desktop\pics2\office_loc.png", 
    r"C:\Users\27118\Desktop\pics2\redkitchen_loc.png"
]

# 设置每帧的宽度和高度
frame_width, frame_height = 640, 480  # 假设每个图片的尺寸相同
output_width = 2001  # 总宽度设置
output_height = 1126  # 总高度设置
num_frames = 290  # 总帧数
fps = 29  # 每秒10帧

# 蓝色和绿色的RGB颜色
blue_color = (0, 0, 255)
green_color = (0, 255, 0)

# 字体设置
font_path = "C:/Windows/Fonts/times.ttf"  # 字体路径
font_size_large = 40
font_captial = ImageFont.truetype(font_path, 60)
font_large = ImageFont.truetype(font_path, 40)
font_normal = ImageFont.truetype(font_path, 30)

# 函数生成过渡效果的一帧，使用向量化操作
def blend_images_with_line(original_img, rendered_img, progress_ratio):
    width, height = original_img.size
    diagonal_position = progress_ratio * (width + height)

    # 转换图像为 NumPy 数组
    original_pixels = np.array(original_img)
    rendered_pixels = np.array(rendered_img)

    # 创建掩码，决定每个像素是来自原图还是渲染图
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    mask = (x + y) < diagonal_position

    # 使用掩码选择原图或渲染图像素
    blended_pixels = np.where(mask[..., None], original_pixels, rendered_pixels)

    # 转回 PIL 图像
    blended_img = Image.fromarray(blended_pixels)

    # 绘制白线，基于 diagonal_position 动态更新线的移动位置
    draw = ImageDraw.Draw(blended_img)
    line_start = (0, diagonal_position) if diagonal_position < height else (diagonal_position - height, height)
    line_end = (diagonal_position, 0) if diagonal_position < width else (width, diagonal_position - width)
    draw.line([line_start, line_end], fill="white", width=3)

    # 绘制蓝色的上边和左边框
    draw.line([(0, 0), (width, 0)], fill=blue_color, width=5)  # 上边
    draw.line([(0, 0), (0, height)], fill=blue_color, width=5)  # 左边

    # 绘制绿色的右边和下边框
    draw.line([(width - 1, 0), (width - 1, height)], fill=green_color, width=5)  # 右边
    draw.line([(0, height - 1), (width, height - 1)], fill=green_color, width=5)  # 下边

    return blended_img

# 函数生成一帧包含三个图像的帧并加上"LOGS"文本和图片标签
def create_video_frame(progress_ratio):
    # 创建一个空白大图像，用于存放三个子图
    combined_frame = Image.new('RGB', (output_width, output_height))
    draw = ImageDraw.Draw(combined_frame)

    # 添加 "LOGS" 文本到图像的上部中央
    draw.text((150, 95), "LOGS", font=font_captial, fill=(255, 255, 255))
    draw.text((150, 95+80), "7-scenes dataset", font=font_large, fill=(255, 255, 255))
    draw.text((150, 95+120), "query image (SfM ground truth pose) v.s. rendered image (estimated pose)", font=font_large, fill=(255, 255, 255))

    # 标签数组
    labels = ["chess", "office", "redkitchen"]

    for i in range(3):
        # 打开原图和渲染图
        original_img = Image.open(original_image_paths[i]).resize((frame_width, frame_height))
        rendered_img = Image.open(rendered_image_paths[i]).resize((frame_width, frame_height))
        
        # 生成包含白线的合成图像
        blended_img = blend_images_with_line(original_img, rendered_img, progress_ratio)

        # 确定子图的位置
        x_offset = i * (output_width // 3)  # 从左到右放置3张图片
        y_offset = (output_height - frame_height + 200) // 2  # 垂直居中放置
        
        # 将合成图像粘贴到大图像中
        combined_frame.paste(blended_img, (x_offset, y_offset))

        # 在每张图片的上方添加对应的标签
        draw.text((x_offset , y_offset - 50), labels[i], font=font_normal, fill=(255, 255, 255))

    return combined_frame

# 创建视频
output_video_path = r"E:/lab/output_video3.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用MP4格式
video = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))

# 生成每一帧视频
for frame_idx in range(num_frames):
    print(f"Processing frame {frame_idx+1}/{num_frames}...")
    progress_ratio = 2 * (frame_idx / num_frames) if frame_idx <= num_frames / 2 else 2 * (1 - frame_idx / num_frames)
    frame_image = create_video_frame(progress_ratio)
    
    # 将PIL图像转换为OpenCV格式并写入视频
    frame_array = np.array(frame_image)
    frame_bgr = cv2.cvtColor(frame_array, cv2.COLOR_RGB2BGR)
    video.write(frame_bgr)

# 释放视频资源
video.release()

print("视频生成完成！")


Processing frame 1/290...
Processing frame 2/290...
Processing frame 3/290...
Processing frame 4/290...
Processing frame 5/290...
Processing frame 6/290...
Processing frame 7/290...
Processing frame 8/290...
Processing frame 9/290...
Processing frame 10/290...
Processing frame 11/290...
Processing frame 12/290...
Processing frame 13/290...
Processing frame 14/290...
Processing frame 15/290...
Processing frame 16/290...
Processing frame 17/290...
Processing frame 18/290...
Processing frame 19/290...
Processing frame 20/290...
Processing frame 21/290...
Processing frame 22/290...
Processing frame 23/290...
Processing frame 24/290...
Processing frame 25/290...
Processing frame 26/290...
Processing frame 27/290...
Processing frame 28/290...
Processing frame 29/290...
Processing frame 30/290...
Processing frame 31/290...
Processing frame 32/290...
Processing frame 33/290...
Processing frame 34/290...
Processing frame 35/290...
Processing frame 36/290...
Processing frame 37/290...
Processing

In [26]:
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

# 使用给定的原图和渲染图路径 (Cambridge Landmarks dataset)
original_image_paths = [
    r"C:\Users\27118\Desktop\pics2\KingsCollege_orig.png", 
    r"C:\Users\27118\Desktop\pics2\StMarysChurch_orig.png"
]

rendered_image_paths = [
    r"C:\Users\27118\Desktop\pics2\KingsCollege_loc.png", 
    r"C:\Users\27118\Desktop\pics2\StMarysChurch_loc.png"
]

# 设置每帧的宽度和高度
frame_width = 853  # 保持宽度不变，调整高度后图像为 853 x 480
frame_height = 480  # 将高度调整为 480
output_width = 2001  # 总宽度设置
output_height = 1126  # 总高度设置
num_frames = 290  # 总帧数
fps = 29  # 每秒10帧

# 蓝色和绿色的RGB颜色
blue_color = (0, 0, 255)
green_color = (0, 255, 0)

# 字体设置
font_path = "C:/Windows/Fonts/times.ttf"  # 字体路径
font_size_large = 40
font_captial = ImageFont.truetype(font_path, 60)
font_large = ImageFont.truetype(font_path, 40)
font_normal = ImageFont.truetype(font_path, 30)

# 函数生成过渡效果的一帧，使用向量化操作
def blend_images_with_line(original_img, rendered_img, progress_ratio):
    width, height = original_img.size
    diagonal_position = progress_ratio * (width + height)

    # 转换图像为 NumPy 数组
    original_pixels = np.array(original_img)
    rendered_pixels = np.array(rendered_img)

    # 创建掩码，决定每个像素是来自原图还是渲染图
    x, y = np.meshgrid(np.arange(width), np.arange(height))
    mask = (x + y) < diagonal_position

    # 使用掩码选择原图或渲染图像素
    blended_pixels = np.where(mask[..., None], original_pixels, rendered_pixels)

    # 转回 PIL 图像
    blended_img = Image.fromarray(blended_pixels)

    # 绘制白线，基于 diagonal_position 动态更新线的移动位置
    draw = ImageDraw.Draw(blended_img)
    line_start = (0, diagonal_position) if diagonal_position < height else (diagonal_position - height, height)
    line_end = (diagonal_position, 0) if diagonal_position < width else (width, diagonal_position - width)
    draw.line([line_start, line_end], fill="white", width=3)

    # 绘制蓝色的上边和左边框
    draw.line([(0, 0), (width, 0)], fill=blue_color, width=5)  # 上边
    draw.line([(0, 0), (0, height)], fill=blue_color, width=5)  # 左边

    # 绘制绿色的右边和下边框
    draw.line([(width - 1, 0), (width - 1, height)], fill=green_color, width=5)  # 右边
    draw.line([(0, height - 1), (width, height - 1)], fill=green_color, width=5)  # 下边

    return blended_img

# 函数生成一帧包含两个图像的帧并加上"LOGS"文本和图片标签
def create_video_frame(progress_ratio):
    # 创建一个空白大图像，用于存放两个子图
    combined_frame = Image.new('RGB', (output_width, output_height))
    draw = ImageDraw.Draw(combined_frame)

    # 添加 "LOGS" 文本到图像的上部中央
    draw.text((150, 95), "LOGS", font=font_captial, fill=(255, 255, 255))
    draw.text((150, 95+80), "Cambridge Landmarks dataset", font=font_large, fill=(255, 255, 255))
    draw.text((150, 95+120), "query image (SfM ground truth pose) v.s. rendered image (estimated pose)", font=font_large, fill=(255, 255, 255))

    # 标签数组
    labels = ["KingsCollege", "StMarysChurch"]

    for i in range(2):
        # 打开原图和渲染图
        original_img = Image.open(original_image_paths[i]).resize((frame_width, frame_height))
        rendered_img = Image.open(rendered_image_paths[i]).resize((frame_width, frame_height))
        
        # 生成包含白线的合成图像
        blended_img = blend_images_with_line(original_img, rendered_img, progress_ratio)

        # 确定子图的位置
        x_offset = i * (output_width // 2) + 75 # 从左到右放置2张图片
        y_offset = (output_height - frame_height + 200) // 2  # 垂直居中放置
        
        # 将合成图像粘贴到大图像中
        combined_frame.paste(blended_img, (x_offset, y_offset))

        # 在每张图片的上方添加对应的标签
        draw.text((x_offset, y_offset - 50), labels[i], font=font_normal, fill=(255, 255, 255))

    return combined_frame

# 创建视频
output_video_path = r"E:/lab/output_video4.mp4"

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # 使用MP4格式
video = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))

# 生成每一帧视频
for frame_idx in range(num_frames):
    print(f"Processing frame {frame_idx+1}/{num_frames}...")
    progress_ratio = 2 * (frame_idx / num_frames) if frame_idx <= num_frames / 2 else 2 * (1 - frame_idx / num_frames)
    frame_image = create_video_frame(progress_ratio)
    
    # 将PIL图像转换为OpenCV格式并写入视频
    frame_array = np.array(frame_image)
    frame_bgr = cv2.cvtColor(frame_array, cv2.COLOR_RGB2BGR)
    video.write(frame_bgr)

# 释放视频资源
video.release()

print("视频生成完成！")


Processing frame 1/290...
Processing frame 2/290...
Processing frame 3/290...
Processing frame 4/290...
Processing frame 5/290...
Processing frame 6/290...
Processing frame 7/290...
Processing frame 8/290...
Processing frame 9/290...
Processing frame 10/290...
Processing frame 11/290...
Processing frame 12/290...
Processing frame 13/290...
Processing frame 14/290...
Processing frame 15/290...
Processing frame 16/290...
Processing frame 17/290...
Processing frame 18/290...
Processing frame 19/290...
Processing frame 20/290...
Processing frame 21/290...
Processing frame 22/290...
Processing frame 23/290...
Processing frame 24/290...
Processing frame 25/290...
Processing frame 26/290...
Processing frame 27/290...
Processing frame 28/290...
Processing frame 29/290...
Processing frame 30/290...
Processing frame 31/290...
Processing frame 32/290...
Processing frame 33/290...
Processing frame 34/290...
Processing frame 35/290...
Processing frame 36/290...
Processing frame 37/290...
Processing

In [28]:
import cv2
import numpy as np

# 路径
cover_image_path = r"C:\Users\27118\Desktop\cover.png"
output_video_path = r"E:\lab\output_video0.avi"

# 视频参数
output_width = 2001
output_height = 1126
fps = 29
duration = 5  # 秒
num_frames = fps * duration  # 总帧数

# 读取并调整封面图片大小
cover_image = cv2.imread(cover_image_path)
resized_cover = cv2.resize(cover_image, (output_width, output_height))

# 创建视频写入器
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (output_width, output_height))

# 生成视频
for _ in range(num_frames):
    out.write(resized_cover)

# 释放资源
out.release()

print("视频 'output_video0.avi' 生成成功！")


视频 'output_video0.avi' 生成成功！


In [34]:
import cv2

# 视频文件路径
video_paths = [
    r"E:\lab\output_video0.avi",
    r"E:\lab\output_video1.avi",
    r"E:\lab\output_video2.avi",
    r"E:\lab\output_video3.mp4",
    r"E:\lab\output_video4.mp4"
]
output_merged_video_path = r"E:\lab\merged_video.mp4"

# 打开 'output_video1.avi' 来获取视频参数
cap = cv2.VideoCapture(video_paths[1])  # 使用 output_video1.avi 获取帧率
fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# 创建输出视频
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_merged_video_path, fourcc, fps, (frame_width, frame_height))

# 合并每个视频
for video_path in video_paths:
    cap = cv2.VideoCapture(video_path)
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        out.write(frame)
    cap.release()

# 释放资源
out.release()
print("所有视频成功合并为 'merged_video.mp4'！")


所有视频成功合并为 'merged_video.mp4'！


In [35]:
import cv2

# 视频文件路径
input_video_path = r"E:\lab\merged_video.mp4"
output_video_path = r"E:\lab\LoGS_video.mp4"
target_fps = 20  # 目标帧率

# 打开输入视频
cap = cv2.VideoCapture(input_video_path)

# 获取输入视频的帧率和尺寸
original_fps = cap.get(cv2.CAP_PROP_FPS)
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# 视频写入器，使用mp4格式
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, target_fps, (frame_width, frame_height))

frame_interval = int(original_fps // target_fps)  # 计算跳过帧的间隔

# 逐帧处理视频
frame_count = 0
while True:
    ret, frame = cap.read()
    
    # 如果没有帧了，退出循环
    if not ret:
        break
    
    # 仅保存每间隔 frame_interval 的帧
    if frame_count % frame_interval == 0:
        out.write(frame)

    frame_count += 1

# 释放资源
cap.release()
out.release()

print(f"视频处理完成，已保存为 {output_video_path}，帧率调整为 {target_fps} FPS。")


视频处理完成，已保存为 E:\lab\LoGS_video.mp4，帧率调整为 20 FPS。
